1. Из ноутбуков по практике "Рекуррентные и одномерные сверточные нейронные сети" выберите лучшую сеть, либо создайте свою.
2. Запустите раздел "Подготовка"
3. Подготовьте датасет с параметрами `VOCAB_SIZE=20'000`, `WIN_SIZE=1000`, `WIN_HOP=100`, как в ноутбуке занятия, и обучите выбранную сеть. Параметры обучения можно взять из практического занятия. Для  всех обучаемых сетей в данной работе они должны быть одни и теже.
4. Поменяйте размер словаря tokenaizera (`VOCAB_SIZE`) на `5000`, `10000`, `40000`.  Пересоздайте датасеты, при этом оставьте `WIN_SIZE=1000`, `WIN_HOP=100`.
Обучите выбранную нейронку на этих датасетах.  Сделайте выводы об  изменении  точности распознавания авторов текстов. Результаты сведите в таблицу
5. Поменяйте длину отрезка текста и шаг окна разбиения текста на векторы  (`WIN_SIZE`, `WIN_HOP`) используя значения (`500`,`50`) и (`2000`,`200`). Пересоздайте датасеты, при этом оставьте `VOCAB_SIZE=20000`. Обучите выбранную нейронку на этих датасетах. Сделайте выводы об  изменении точности распознавания авторов текстов.

Результаты всей работы сведите в таблицу.

## Подготовка

### Импорты

In [ ]:
# Функции операционной системы
import os

# Регулярные выражения
import re

# Работа со временем
import time

# Для работы с архивами
import zipfile

# Загрузка датасетов из облака google
import gdown

# Работа с массивами данных
import numpy as np
import pandas as pd

# Функция display для отображения изображений в Jupyter Notebook
from IPython.display import display

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Основные слои
from tensorflow.keras.layers import (
    BatchNormalization,
    Conv1D,
    Dense,
    Dropout,
    Embedding,
    Flatten,
    MaxPooling1D,
    SpatialDropout1D,
)

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

%matplotlib inline

### Загрузка и распаковка архива

In [2]:
if not os.path.exists('writers.zip'):
    # Загрузим датасет из облака
    gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l7/writers.zip', None, quiet=True)

In [3]:
# Распакуем архив в папку writers
# !unzip -o writers.zip -d writers/
if not os.path.exists('writers'):
    with zipfile.ZipFile('writers.zip', 'r') as zip_ref:
        zip_ref.extractall('writers')

# Выводим список файлов в папке writers
print("\n".join(os.listdir('writers')))

(Стругацкие) Тестовая_2 вместе.txt
(Рэй Брэдберри) Обучающая_22 вместе.txt
(О. Генри) Обучающая_50 вместе.txt
(Булгаков) Тестовая_2 вместе.txt
(Булгаков) Обучающая_5 вместе.txt
(Макс Фрай) Обучающая_5 вместе.txt
(Клиффорд_Саймак) Обучающая_5 вместе.txt
(О. Генри) Тестовая_20 вместе.txt
(Рэй Брэдберри) Тестовая_8 вместе.txt
(Макс Фрай) Тестовая_2 вместе.txt
(Стругацкие) Обучающая_5 вместе.txt
(Клиффорд_Саймак) Тестовая_2 вместе.txt


### Подготовка данных

In [5]:
# Настройка констант для загрузки данных
FILE_DIR  = 'writers'                     # Папка с текстовыми файлами
SIG_TRAIN = 'обучающая'                   # Признак обучающей выборки в имени файла
SIG_TEST  = 'тестовая'                    # Признак тестовой выборки в имени файла

In [ ]:
# Подготовим пустые списки

CLASS_LIST = []  # Список классов
text_train = []  # Список для оучающей выборки
text_test = []  # Список для тестовой выборки

# Получим списка файлов в папке
file_list = os.listdir(FILE_DIR)

for file_name in file_list:
    # Выделяем имя класса и типа выборки из имени файла
    m = re.match("\((.+)\) (\S+)_", file_name)
    # Если выделение получилось, то файл обрабатываем
    if m:
        # Получим имя класса
        class_name = m[1]

        # Получим имя выборки
        subset_name = m[2].lower()

        # Проверим тип выборки
        is_train = SIG_TRAIN in subset_name
        is_test = SIG_TEST in subset_name

        # Если тип выборки обучающая либо тестовая - файл обрабатываем
        if is_train or is_test:
            # Добавляем новый класс, если его еще нет в списке
            if class_name not in CLASS_LIST:
                print(f'Добавление класса "{class_name}"')
                CLASS_LIST.append(class_name)

                # Инициализируем соответствующих классу строки текста
                text_train.append("")
                text_test.append("")

            # Найдем индекс класса для добавления содержимого файла в выборку
            cls = CLASS_LIST.index(class_name)
            print(
                f'Добавление файла "{file_name}" в класс '
                f'"{CLASS_LIST[cls]}", {subset_name} выборка.'
            )

            # Откроем файл на чтение
            with open(f"{FILE_DIR}/{file_name}", "r") as f:
                # Загрузим содержимого файла в строку
                text = f.read()
            # Определим выборку, куда будет добавлено содержимое
            subset = text_train if is_train else text_test

            # Добавим текста к соответствующей выборке класса. Концы строк заменяются на пробел
            subset[cls] += " " + text.replace("\n", " ")

Добавление класса "Стругацкие"
Добавление файла "(Стругацкие) Тестовая_2 вместе.txt" в класс "Стругацкие", тестовая выборка.
Добавление класса "Рэй Брэдберри"
Добавление файла "(Рэй Брэдберри) Обучающая_22 вместе.txt" в класс "Рэй Брэдберри", обучающая выборка.
Добавление класса "О. Генри"
Добавление файла "(О. Генри) Обучающая_50 вместе.txt" в класс "О. Генри", обучающая выборка.
Добавление класса "Булгаков"
Добавление файла "(Булгаков) Тестовая_2 вместе.txt" в класс "Булгаков", тестовая выборка.
Добавление файла "(Булгаков) Обучающая_5 вместе.txt" в класс "Булгаков", обучающая выборка.
Добавление класса "Макс Фрай"
Добавление файла "(Макс Фрай) Обучающая_5 вместе.txt" в класс "Макс Фрай", обучающая выборка.
Добавление класса "Клиффорд_Саймак"
Добавление файла "(Клиффорд_Саймак) Обучающая_5 вместе.txt" в класс "Клиффорд_Саймак", обучающая выборка.
Добавление файла "(О. Генри) Тестовая_20 вместе.txt" в класс "О. Генри", тестовая выборка.
Добавление файла "(Рэй Брэдберри) Тестовая_8 вме

In [7]:
# Определим количество классов
CLASS_COUNT = len(CLASS_LIST)

In [8]:
# Выведем прочитанные классы текстов
print(CLASS_LIST)

['Стругацкие', 'Рэй Брэдберри', 'О. Генри', 'Булгаков', 'Макс Фрай', 'Клиффорд_Саймак']


In [9]:
# Посчитаем количество текстов в обучающей выборке
print(len(text_train))

6


In [10]:
# Проверим загрузки: выведем начальные отрывки из каждого класса

for cls in range(CLASS_COUNT):                   # Запустим цикл по числу классов
    print(f'Класс: {CLASS_LIST[cls]}')           # Выведем имя класса
    print(f'  train: {text_train[cls][:200]}')   # Выведем фрагмент обучающей выборки
    print(f'  test : {text_test[cls][:200]}')    # Выведем фрагмент тестовой выборки
    print()

Класс: Стругацкие
  train:  Парень из преисподней     1     Ну и деревня! Сроду я таких деревень не видел и не знал даже, что такие деревни бывают. Дома круглые, бурые, без окон, торчат на сваях, как сторожевые вышки, а под ним
  test :  ﻿ОТЕЛЬ «У ПОГИБШЕГО АЛЬПИНИСТА»    ГЛАВА 1     Я остановил машину, вылез и снял черные очки. Все было так, как рассказывал Згут. Отель был двухэтажный, желтый с зеленым, над крыльцом красовалась трау

Класс: Рэй Брэдберри
  train:  ﻿451° по Фаренгейту   ДОНУ КОНГДОНУ С БЛАГОДАРНОСТЬЮ   Если тебе дадут линованную бумагу, пиши поперёк.  Хуан Рамон Хименес   Часть 1  ОЧАГ И САЛАМАНДРА   Жечь было наслаждением. Какое-то особое насл
  test :  ﻿Марсианские хроники   МОЕЙ ЖЕНЕ МАРГАРЕТ С ИСКРЕННЕЙ ЛЮБОВЬЮ   «Великое дело – способность удивляться, – сказал философ. – Космические полеты снова сделали всех нас детьми».   Январь 1999  Ракетное 

Класс: О. Генри
  train:  «Лиса-на-рассвете»   Коралио нежился в полуденном зное, как томная красавица в сурово хранимо

### Контекстный менеджер

In [11]:
# Контекстный менеджер для измерения времени операций
# Операция обертывается менеджером с помощью оператора with

class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        print('Время обработки: {:.2f} с'.format(time.time() - self.t))

## Решение

### Списки параметров токенизации

In [12]:
VOCAB_SIZES=(5000, 10000, 20000, 40000)
WIN_PARAMS=((500, 50), (1000, 100), (2000, 200))

### Функции для деления последовательностей на окна

In [13]:
# Функция разбиения последовательности на отрезки скользящим окном
# На входе - последовательность индексов, размер окна, шаг окна
def split_sequence(sequence, win_size, hop):
    # Последовательность разбивается на части до последнего полного окна
    return [
        sequence[i : i + win_size] for i in range(0, len(sequence) - win_size + 1, hop)
    ]


# Функция формирования выборок из последовательностей индексов
# формирует выборку отрезков и соответствующих им меток классов в виде one hot encoding
def vectorize_sequence(seq_list, win_size, hop):
    # В списке последовательности следуют в порядке их классов
    # Всего последовательностей в списке ровно столько, сколько классов
    class_count = len(seq_list)

    # Списки для исходных векторов и категориальных меток класса
    x, y = [], []

    # Для каждого класса:
    for cls in range(class_count):
        # Разбиение последовательности класса cls на отрезки
        vectors = split_sequence(seq_list[cls], win_size, hop)
        # Добавление отрезков в выборку
        x += vectors
        # Для всех отрезков класса cls добавление меток класса в виде OHE
        y += [utils.to_categorical(cls, class_count)] * len(vectors)

    # Возврат результатов как numpy-массивов
    return np.array(x), np.array(y)


### Функция создания выборок и модели, обучение модели на созданных выборках

In [ ]:
# Функция подготовки данных, создания и обучения нейронной сети
def preparation_and_creation_ns(
    models_list, seq_train, seq_test, vocab_size, win_size, win_hope
):
    with timex():  # Подготовка выборок
        x_train, y_train = vectorize_sequence(seq_train, win_size, win_hope)
        x_test, y_test = vectorize_sequence(seq_test, win_size, win_hope)

    model = Sequential(  # Создание модели нейронной сети
        [
            Embedding(vocab_size, 10),
            SpatialDropout1D(0.2),
            BatchNormalization(),
            Conv1D(20, 5, activation="relu", padding="same"),
            Conv1D(20, 5, activation="relu"),
            MaxPooling1D(2),
            Dropout(0.2),
            BatchNormalization(),
            Flatten(),
            Dense(CLASS_COUNT, activation="softmax"),
        ]
    )

    model.compile(  # Компиляция модели
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    history = model.fit(  # Обучение модели
        x_train,
        y_train,
        epochs=100,
        batch_size=256,
        validation_data=(x_test, y_test),
        verbose=1,
    )

    models_list.append(  # Добавление параметров модели в список
        {
            "VOCAB_SIZE": vocab_size,
            "WIN_SIZE": win_size,
            "WIN_HOPE": win_hope,
            "Точность": history.history["accuracy"][-1],
            "Точность на проверочной выборке": history.history["val_accuracy"][-1],
        }
    )

### Цикл итерации по VOCAB_SIZES, 
### и вложенный цикл итерации по WIN_PARAMS при vocab_size=20000

In [ ]:
models_vocab_list = []  # Список для моделей с разными размерами словаря
models_win_params_list = []  # Список для моделей с разными размерами окна и шагом

# Цикл по размерам словаря
for vocab_size in VOCAB_SIZES:
    with timex():
        # Создание токенизатора
        tokenizer = Tokenizer(
            num_words=vocab_size,
            filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
            lower=True,
            split=" ",
            oov_token="неизвестное_слово",
            char_level=False,
        )
        tokenizer.fit_on_texts(text_train)  # Обучение токенизатора на обучающей выборке
        # Преобразование текстов в последовательности индексов
        seq_train = tokenizer.texts_to_sequences(text_train)
        seq_test = tokenizer.texts_to_sequences(text_test)

    if vocab_size != 20000:
        # Для размеров словарей, отличных от 20000
        # используем только одно окно = (1000, 100)
        win_size, win_hope = WIN_PARAMS[1]
        preparation_and_creation_ns(  # Подготовка и создание нейронной сети
            models_vocab_list, seq_train, seq_test, vocab_size, win_size, win_hope
        )
    else:
        for win_size, win_hope in WIN_PARAMS:
            # Для размера словаря 20000 используем все окна
            preparation_and_creation_ns(
                models_win_params_list,
                seq_train,
                seq_test,
                vocab_size,
                win_size,
                win_hope,
            )
        # Добавляем параметры модели с размером словаря 20000 и окном (1000, 100)
        # в первый список для сравнения с другими размерами словаря
        models_vocab_list.append(models_win_params_list[1])


Время обработки: 3.33 с
Время обработки: 0.80 с


Epoch 1/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 12s 76ms/step - accuracy: 0.3215 - loss: 1.9572 - val_accuracy: 0.1253 - val_loss: 2.3560
Epoch 2/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8158 - loss: 0.4899 - val_accuracy: 0.0782 - val_loss: 5.3864
Epoch 3/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9779 - loss: 0.0773 - val_accuracy: 0.0981 - val_loss: 6.7929
Epoch 4/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9935 - loss: 0.0279 - val_accuracy: 0.1971 - val_loss: 7.4754
Epoch 5/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9935 - loss: 0.0226 - val_accuracy: 0.1971 - val_loss: 7.4412
Epoch 6/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9966 - loss: 0.0129 - val_accuracy: 0.2022 - val_loss: 5.3152
Epoch 7/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9969 - loss: 0.0100 - val_accuracy: 0.3494 - val_loss: 3.4185
Epoch 8/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9946 - loss: 0.0158 - val_accuracy: 0

### Вывод сравнительных таблиц

In [ ]:
# Создание сравнительных таблиц
df1 = pd.DataFrame(models_vocab_list)
df2 = pd.DataFrame(models_win_params_list)

In [ ]:
display(df1)

,VOCAB_SIZE,WIN_SIZE,WIN_HOPE,Точность,Точность на проверочной выборке
0,5000,1000,100,0.999717,0.605444
1,10000,1000,100,0.999263,0.577625
2,20000,1000,100,0.998753,0.615465
3,40000,1000,100,0.999490,0.608286


In [ ]:
display(df2)

,VOCAB_SIZE,WIN_SIZE,WIN_HOPE,Точность,Точность на проверочной выборке
0,20000,500,50,0.999547,0.562672
1,20000,1000,100,0.998753,0.615465
2,20000,2000,200,0.998066,0.466204
